Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [4]:
NAME = "ROHITH KUMAR POSHALA"
COLLABORATORS = ""

---

# Lab 2 Introduction


In this lab you will use parse a vcf file to extracts parts of it and load it into a database. 
The fields you will be parsing from the vcf file are: 
```
CHROM
POS	
ID	
REF	
ALT	
QUAL	
FILTER
```
and from the INFO column, the following fields:
```
1000g2015aug_all
ExAC_ALL
FATHMM_pred
LRT_pred
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm-MKL_coding_pred.
```
The fields:
```
FATHMM_pred
LRT_pred
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm-MKL_coding_pred
```
are predictor fields. They use a letter to indicate whether a given variation is harmful or not.

NOTE: use the helper functions in cell2. To use them, you have to run CELL 2! 


In [5]:
## Helper functions

import os
import sqlite3
from sqlite3 import Error
import gzip

def create_connection(db_file, delete_db=False):
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)



# Part 1 (10 pts)

In part1 you will open the gzipped vcf file using the python gzip module and figure out all the possible values for the predictor fields. 

You do not have to unzip the file. Use `with gzip.open(filename,'rt') as fp:` to read one line at a time. 



In [6]:
def get_predictor_values(filename):
    """
    See part 1 description 
    """
    import gzip

   
    keys_save_value = [
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ]

    # YOUR CODE HERE
    with gzip.open(filename,'rt') as fp :
        header=[]
        line=[]
        for l in fp:
            if l.startswith('##'):
                continue # lines starting with double hash are skipped
            else:
                if l.startswith('#'):
                    header.append(l.strip()[1:])
                else:
                    line.append(l.strip())                   
    index = [header[0].split("\t").index(i) for i in header[0].split("\t") if i == 'INFO'][0]
    predictor_dict = {}
    key_list = []
    for j in line :
        llist = j.split("\t")
        info_list = llist[index].split(";")
        for k in info_list:
            if '=' in k :
                int_list = []
                k_v_list = k.split("=")
                if k_v_list[0] in keys_save_value :
                    if k_v_list[0] not in key_list:
                        key_list.append(k_v_list[0])
                        if k_v_list[1] != '.':
                            int_list.append(k_v_list[1])
                        predictor_dict.update({k_v_list[0]:int_list})
                    else :
                        if k_v_list[1] != '.' and k_v_list[1] not in predictor_dict[k_v_list[0]]:
                            predictor_dict[k_v_list[0]].append(k_v_list[1])
    return predictor_dict

    raise NotImplementedError()


In [7]:
# filename = 'test_4families_annovar.vcf.gz'
# predictor_values = get_predictor_values(filename)
# from pprint import pprint
# pprint(predictor_values)

In [8]:

## Can take 30 seconds to run!
expected_solution = {'SIFT_pred': ['D', 'T'], 'Polyphen2_HDIV_pred': ['D', 'B', 'P'], 'Polyphen2_HVAR_pred': ['D', 'B', 'P'], 'LRT_pred': ['D', 'N', 'U'], 'MutationTaster_pred': ['D', 'P', 'N', 'A'], 'MutationAssessor_pred': ['H', 'N', 'L', 'M'], 'FATHMM_pred': ['T', 'D'], 'PROVEAN_pred': ['D', 'N'], 'MetaSVM_pred': ['D', 'T'], 'MetaLR_pred': ['D', 'T'], 'fathmm-MKL_coding_pred': ['D', 'N']}
filename = 'test_4families_annovar.vcf.gz'
predictor_values = get_predictor_values(filename)
assert predictor_values == expected_solution



# Part 2 (No points)

You will now create 13 tables
Tables 1-11 will be for :

FATHMM_pred
LRT_pred 
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm_MKL_coding_pred ## NOTE: I have replaced the dash with an underscore. 

The first column for each of these tables will be the name of the field + ID, e.g., FATHMM_predID. This
column will be of type not null primary key. The second column will be called `prediction` and will be of
type TEXT not null. The prediction values will be values you extracted for each of the fields in the previous step. 
For example, 'FATHMM_pred' has two prediction values 'T' and 'D'. Name the tables after their field name, e.g.,  
call table that will contains values of FATHMM_pred `FATHMM_pred`.  Make sure to sort the values, meaning, D should 
be inserted before T. 


Table 12 will be the Variants table. 
The first column will be VariantID. 

The other columns will be:
CHROM   
POS 
ID  
REF 
ALT 
QUAL    
FILTER
thousandg2015aug_all ##NOTE: a column name cannot start with a number, so you have to rename!
ExAC_ALL


Table 12 will have 11 more columns that relate to each of prediction table. Consider writing a utility function
to fetch the primary key for a given prediction from each of the prediction table. Name each of the 
column should be the name of predictor + ID, e.g., FATHMM_predID. 


You have already deteremined their data type, so use that to set their data type. 
For integer use INTEGER. 
For float use REAL. 
For string use TEXT. 

Table 13  will be called PredictionStats. The first column will be PredictorStatsID INTEGER NOT NULL PRIMARY KEY. 
The second column will be VariantID. The third column will be PredictorName. The fourth column will be PredictorValue. 
This is not a normalized table!

The prediction value will be a float value mapped from the prediction text.  Use the following information to 
assign values: 
REF: https://brb.nci.nih.gov/seqtools/colexpanno.html#dbnsfp

FATHMM_pred
- T = 0
- D = 1

LRT_pred 
- D = 1
- N = 0
- U = 0

MetaLR_pred
- T = 0
- D = 1

MetaSVM_pred
- T = 0
- D = 1

MutationAssessor_pred
- H = 1
- N = 0
- L = 0.25
- M = 0.5

MutationTaster_pred
- D = 1
- P = 0
- N = 0
- A = 1

PROVEAN_pred
- D = 1
- N = 0

Polyphen2_HDIV_pred
- D = 1
- B = 0
- P = 0.5

Polyphen2_HVAR_pred
- D = 1
- B = 0
- P = 0.5

SIFT_pred
- D = 1
- T = 0

fathmm-MKL_coding_pred
- D = 1
- N = 0


```

prediction_mapping = {
    'FATHMM_pred': {'T': 0, 'D': 1},
    'MetaLR_pred': {'T': 0, 'D': 1},
    'MetaSVM_pred': {'T': 0, 'D': 1},
    'SIFT_pred': {'T': 0, 'D': 1},
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 0},
    'LRT_pred': {'U': 0, 'N': 0, 'D': 1},
    'MutationAssessor_pred': {'H': 1, 'N': 0, 'L': 0.25, 'M': 0.5},  
    'MutationTaster_pred': {'D': 1, 'P': 0, 'N': 0, 'A': 1},
    'PROVEAN_pred': {'D': 1, 'N': 0},
    'Polyphen2_HDIV_pred': {'D': 1, 'B': 0, 'P': 0.5},
    'Polyphen2_HVAR_pred': {'D': 1, 'B': 0, 'P': 0.5},
}


```


The idea is that 11 predictors have been used to annotate all the variants in the file. This table combines all that information in one table. 
By grouping the table based on variantid and summing the prediction values mapped from the prediction text, you can find which variants have a consensus on their being deterimental. 

IMPORTANT: instead of fathmm-MKL_coding_pred use fathmm_MKL_coding_pred everywhere. 
HINT: You have to commit the changes you make to the table, otherwise your changes will not be saved. Consider wrapping all changes inside `with conn:`




In [9]:
# Creating lab2.db 
db_file = 'lab2.db'
conn = create_connection(db_file, delete_db=True)
conn.close()


In [10]:
def create_tables_1_11(db_file):
    """
    Create tables 1 to 11.
    
    INPUT: db_file -- Name of database file -- eg. lab2.db
    
    """
    # YOUR CODE HERE
    conn = create_connection(db_file)
    with conn :
        for key,value in predictor_values.items() : 
            if '-' in key :
                key = key.replace('-','_')
            create_table_sql = 'CREATE TABLE ' + key +' ('+ key +'ID INTEGER NOT NULL PRIMARY KEY, ' + 'prediction text NOT NULL);'
            create_table(conn, create_table_sql)
            value.sort()
            c = 0
            for j in value :
                c += 1
                insertsql = 'INSERT INTO ' + key +' ('+ key +'ID , ' + 'prediction) VALUES (?, ?)'
                cur = conn.cursor()
                cur.execute(insertsql,(c,j))
                c = cur.lastrowid
            
         

    
#     raise NotImplementedError()

db_file = 'lab2.db'    
create_tables_1_11(db_file)

In [11]:
# conn = create_connection('lab2.db')
# with conn :
#     a = """select * from fathmm_MKL_coding_pred;"""
#     cur = conn.cursor()
#     cur.execute(a)
#     rows = cur.fetchall()
#     for row in rows:
#         print(row)    
#     rows = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

# #     rows = cur.fetchall()
#     for row in rows:
#         print(row)

# Part 3 (10 pts)

Write a function that returns a dictionary that maps for a given predictor its letter prediction to foreign key value. 
Conver the dash in 'fathmm-MKL_coding_pred' to an underscore. 


In [12]:
def get_predictor_value_to_fk_map(db_file):
    """
    See part 3 description 
    
    INPUT: db_file -- Name of database file -- eg. lab2.db
    
    """
    # YOUR CODE HERE
    conn = create_connection(db_file)
    with conn :
        dic = {}
        for key in predictor_values.keys():
            if '-' in key :
                key = key.replace('-','_')
            cur = conn.cursor()
            selectsql = "SELECT * FROM "+ key +";"
            cur.execute(selectsql)
            rows = cur.fetchall()
            indic = {}
            for row in rows:
                indic.update({row[1] : row[0]})
#                 print(row)
            dic.update({key : indic})

    return dic
    
#     raise NotImplementedError()


In [13]:
# db_file = 'lab2.db'
# predictor_fk_map = get_predictor_value_to_fk_map(db_file)
# from pprint import pprint
# pprint(predictor_fk_map)

In [14]:
expected_solution = {
    'FATHMM_pred': {'D': 1, 'T': 2}, 
    'LRT_pred': {'D': 1, 'N': 2, 'U': 3}, 
    'MetaLR_pred': {'D': 1, 'T': 2}, 
    'MetaSVM_pred': {'D': 1, 'T': 2}, 
    'MutationAssessor_pred': {'H': 1, 'L': 2, 'M': 3, 'N': 4}, 
    'MutationTaster_pred': {'A': 1, 'D': 2, 'N': 3, 'P': 4}, 
    'PROVEAN_pred': {'D': 1, 'N': 2}, 
    'Polyphen2_HDIV_pred': {'B': 1, 'D': 2, 'P': 3}, 
    'Polyphen2_HVAR_pred': {'B': 1, 'D': 2, 'P': 3}, 
    'SIFT_pred': {'D': 1, 'T': 2}, 
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 2}}

db_file = 'lab2.db'
predictor_fk_map = get_predictor_value_to_fk_map(db_file)
assert predictor_fk_map == expected_solution



# Part 4 (No Points)
Create table 12 or the variants table. See description above


In [15]:
def create_variants_table(db_file):
    """
    Part 4
    """
    # YOUR CODE HERE
    variant_list = ['CHROM','POS','ID','REF','ALT','QUAL','FILTER','thousandg2015aug_all','ExAC_ALL']
    conn = create_connection(db_file)
    with conn :
        create_table_sql = '''CREATE TABLE Variants ( VariantID INTEGER NOT NULL PRIMARY KEY)'''
        create_table(conn, create_table_sql)
        for i in variant_list:
#              or i == 'CHROM'
            if i == 'POS':
                column_type = 'INTEGER'
            elif i == 'QUAL' or i == 'thousandg2015aug_all' or i == 'ExAC_ALL':
                column_type = 'REAL'
            else :
                column_type = 'TEXT'
            cur = conn.cursor()
            cur.execute("ALTER TABLE Variants ADD COLUMN '{cn}' {ct}".format(cn=i, ct=column_type))
        key_list = [i for i in predictor_values.keys()]
        key_list.sort()
        for key in key_list : 
            if '-' in key :
                key = key.replace('-','_')
            cur = conn.cursor()
            cur.execute('PRAGMA TABLE_INFO({})'.format(key))
            table_schema = cur.fetchall()
            column_names = [tup[1] for tup in table_schema]
            column_types = [tup[2] for tup in table_schema]
            cur.execute("ALTER TABLE Variants ADD COLUMN '{cn}' {ct}".format(cn=column_names[0], ct=column_types[0]))
        
#     raise NotImplementedError()

# create table
db_file = 'lab2.db'
create_variants_table(db_file)


In [16]:
# conn = create_connection('lab2.db')
# with conn :
#     cur = conn.cursor()
# #     rows = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

# #     rows = cur.fetchall()
# #     for row in rows:
# #         print(row)
#     d = 'Variants'
#     cur.execute('PRAGMA TABLE_INFO({})'.format(d))
#     sd = cur.fetchall()
#     column_names = [tup[1] for tup in sd]
#     column_types = [tup[2] for tup in sd]
# #     a = """select * from Variants;"""
# #     cur = conn.cursor()
# #     cur.execute(a)
#     rows = cur.fetchall()
#     for row in rows:
#         print(row)    
#     print(column_names)
#     print(column_types)

# Part 5 (No Points)

Create table 13 -- or the prediction stats table. See description above. 


In [17]:
def create_predictionstats_table(db_file):
    """
    Part 5   
    """
    # YOUR CODE HERE
    conn = create_connection(db_file)
    with conn :
        create_table_sql = '''CREATE TABLE PredictionStats ( PredictorStatsID INTEGER NOT NULL PRIMARY KEY , VariantID INTEGER , PredictorName TEXT, PredictorValue REAL )'''
        create_table(conn, create_table_sql) 
    
#     raise NotImplementedError()
    
db_file = 'lab2.db'
create_predictionstats_table(db_file)

# Part 6 (10 Points)

Write a function to pull the following info fields given the whole info field. 
```
values_to_pull = [
        '1000g2015aug_all',
        'ExAC_ALL',
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ]
```


In [18]:
def pull_info_values(info):
    """
    See part 6 description
    """
    # YOUR CODE HERE
    values_to_pull = [
        '1000g2015aug_all',
        'ExAC_ALL',
        'FATHMM_pred',
        'LRT_pred',
        'MetaLR_pred',
        'MetaSVM_pred',
        'MutationAssessor_pred',
        'MutationTaster_pred',
        'PROVEAN_pred',
        'Polyphen2_HDIV_pred',
        'Polyphen2_HVAR_pred',
        'SIFT_pred',
        'fathmm-MKL_coding_pred',
    ]
    
    dic_info_values = {}
    info_list = info.split(";")
    for j in info_list:
        j.strip()
        if '=' in j :
            k_v_list = j.split("=")
            if k_v_list[0] in values_to_pull :
                if '1000' in k_v_list[0] :
                    k_v_list[0] = k_v_list[0].replace('1000','thousand')
                elif '-' in k_v_list[0] :
                    k_v_list[0] = k_v_list[0].replace('-','_')
#                     k_v_list[0] = 'thousandg2015aug_all'
                dic_info_values.update({k_v_list[0]:k_v_list[1]})
    return dic_info_values
    
#     raise NotImplementedError()



In [19]:
# sample_info_input = "AC=2;AF=0.333;AN=6;BaseQRankSum=2.23;ClippingRankSum=0;DP=131;ExcessHet=3.9794;FS=2.831;MLEAC=2;MLEAF=0.333;MQ=60;MQRankSum=0;QD=12.06;ReadPosRankSum=-0.293;SOR=0.592;VQSLOD=21.79;culprit=MQ;DB;POSITIVE_TRAIN_SITE;ANNOVAR_DATE=2018-04-16;Func.refGene=exonic;Gene.refGene=MAST2;GeneDetail.refGene=.;ExonicFunc.refGene=nonsynonymous_SNV;AAChange.refGene=MAST2:NM_015112:exon29:c.G3910A:p.V1304M;Func.ensGene=exonic;Gene.ensGene=ENSG00000086015;GeneDetail.ensGene=.;ExonicFunc.ensGene=nonsynonymous_SNV;AAChange.ensGene=ENSG00000086015:ENST00000361297:exon29:c.G3910A:p.V1304M;cytoBand=1p34.1;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=0.034;SIFT_converted_rankscore=0.440;SIFT_pred=D;Polyphen2_HDIV_score=0.951;Polyphen2_HDIV_rankscore=0.520;Polyphen2_HDIV_pred=P;Polyphen2_HVAR_score=0.514;Polyphen2_HVAR_rankscore=0.462;Polyphen2_HVAR_pred=P;LRT_score=0.002;LRT_converted_rankscore=0.368;LRT_pred=N;MutationTaster_score=1.000;MutationTaster_converted_rankscore=0.810;MutationTaster_pred=D;MutationAssessor_score=1.67;MutationAssessor_score_rankscore=0.430;MutationAssessor_pred=L;FATHMM_score=1.36;FATHMM_converted_rankscore=0.344;FATHMM_pred=T;PROVEAN_score=-1.4;PROVEAN_converted_rankscore=0.346;PROVEAN_pred=N;VEST3_score=0.158;VEST3_rankscore=0.189;MetaSVM_score=-1.142;MetaSVM_rankscore=0.013;MetaSVM_pred=T;MetaLR_score=0.008;MetaLR_rankscore=0.029;MetaLR_pred=T;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=4.716;CADD_raw_rankscore=0.632;CADD_phred=24.6;DANN_score=0.998;DANN_rankscore=0.927;fathmm-MKL_coding_score=0.900;fathmm-MKL_coding_rankscore=0.506;fathmm-MKL_coding_pred=D;Eigen_coding_or_noncoding=c;Eigen-raw=0.461;Eigen-PC-raw=0.469;GenoCanyon_score=1.000;GenoCanyon_score_rankscore=0.747;integrated_fitCons_score=0.672;integrated_fitCons_score_rankscore=0.522;integrated_confidence_value=0;GERP++_RS=4.22;GERP++_RS_rankscore=0.490;phyloP100way_vertebrate=4.989;phyloP100way_vertebrate_rankscore=0.634;phyloP20way_mammalian=1.047;phyloP20way_mammalian_rankscore=0.674;phastCons100way_vertebrate=1.000;phastCons100way_vertebrate_rankscore=0.715;phastCons20way_mammalian=0.999;phastCons20way_mammalian_rankscore=0.750;SiPhy_29way_logOdds=17.151;SiPhy_29way_logOdds_rankscore=0.866;Interpro_domain=.;GTEx_V6_gene=ENSG00000162415.6;GTEx_V6_tissue=Nerve_Tibial;esp6500siv2_all=0.0560;esp6500siv2_aa=0.0160;esp6500siv2_ea=0.0761;ExAC_ALL=0.0553;ExAC_AFR=0.0140;ExAC_AMR=0.0386;ExAC_EAS=0.0005;ExAC_FIN=0.0798;ExAC_NFE=0.0788;ExAC_OTH=0.0669;ExAC_SAS=0.0145;ExAC_nontcga_ALL=0.0541;ExAC_nontcga_AFR=0.0129;ExAC_nontcga_AMR=0.0379;ExAC_nontcga_EAS=0.0004;ExAC_nontcga_FIN=0.0798;ExAC_nontcga_NFE=0.0802;ExAC_nontcga_OTH=0.0716;ExAC_nontcga_SAS=0.0144;ExAC_nonpsych_ALL=0.0496;ExAC_nonpsych_AFR=0.0140;ExAC_nonpsych_AMR=0.0386;ExAC_nonpsych_EAS=0.0005;ExAC_nonpsych_FIN=0.0763;ExAC_nonpsych_NFE=0.0785;ExAC_nonpsych_OTH=0.0638;ExAC_nonpsych_SAS=0.0145;1000g2015aug_all=0.024361;1000g2015aug_afr=0.0038;1000g2015aug_amr=0.0461;1000g2015aug_eur=0.0795;1000g2015aug_sas=0.0041;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=rs33931638;avsnp150=rs33931638;CADD13_RawScore=4.716301;CADD13_PHRED=24.6;Eigen=0.4614;REVEL=0.098;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0507;gnomAD_genome_AFR=0.0114;gnomAD_genome_AMR=0.0430;gnomAD_genome_ASJ=0.1159;gnomAD_genome_EAS=0;gnomAD_genome_FIN=0.0802;gnomAD_genome_NFE=0.0702;gnomAD_genome_OTH=0.0695;gerp++gt2=4.22;cosmic70=.;InterVar_automated=Benign;PVS1=0;PS1=0;PS2=0;PS3=0;PS4=0;PM1=0;PM2=0;PM3=0;PM4=0;PM5=0;PM6=0;PP1=0;PP2=0;PP3=0;PP4=0;PP5=0;BA1=1;BS1=1;BS2=0;BS3=0;BS4=0;BP1=0;BP2=0;BP3=0;BP4=0;BP5=0;BP6=0;BP7=0;Kaviar_AF=0.0552127;Kaviar_AC=8536;Kaviar_AN=154602;ALLELE_END"

# solution  = pull_info_values(sample_info_input)
# from pprint import pprint
# pprint(solution)

In [20]:
sample_info_input = "AC=2;AF=0.333;AN=6;BaseQRankSum=2.23;ClippingRankSum=0;DP=131;ExcessHet=3.9794;FS=2.831;MLEAC=2;MLEAF=0.333;MQ=60;MQRankSum=0;QD=12.06;ReadPosRankSum=-0.293;SOR=0.592;VQSLOD=21.79;culprit=MQ;DB;POSITIVE_TRAIN_SITE;ANNOVAR_DATE=2018-04-16;Func.refGene=exonic;Gene.refGene=MAST2;GeneDetail.refGene=.;ExonicFunc.refGene=nonsynonymous_SNV;AAChange.refGene=MAST2:NM_015112:exon29:c.G3910A:p.V1304M;Func.ensGene=exonic;Gene.ensGene=ENSG00000086015;GeneDetail.ensGene=.;ExonicFunc.ensGene=nonsynonymous_SNV;AAChange.ensGene=ENSG00000086015:ENST00000361297:exon29:c.G3910A:p.V1304M;cytoBand=1p34.1;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=0.034;SIFT_converted_rankscore=0.440;SIFT_pred=D;Polyphen2_HDIV_score=0.951;Polyphen2_HDIV_rankscore=0.520;Polyphen2_HDIV_pred=P;Polyphen2_HVAR_score=0.514;Polyphen2_HVAR_rankscore=0.462;Polyphen2_HVAR_pred=P;LRT_score=0.002;LRT_converted_rankscore=0.368;LRT_pred=N;MutationTaster_score=1.000;MutationTaster_converted_rankscore=0.810;MutationTaster_pred=D;MutationAssessor_score=1.67;MutationAssessor_score_rankscore=0.430;MutationAssessor_pred=L;FATHMM_score=1.36;FATHMM_converted_rankscore=0.344;FATHMM_pred=T;PROVEAN_score=-1.4;PROVEAN_converted_rankscore=0.346;PROVEAN_pred=N;VEST3_score=0.158;VEST3_rankscore=0.189;MetaSVM_score=-1.142;MetaSVM_rankscore=0.013;MetaSVM_pred=T;MetaLR_score=0.008;MetaLR_rankscore=0.029;MetaLR_pred=T;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=4.716;CADD_raw_rankscore=0.632;CADD_phred=24.6;DANN_score=0.998;DANN_rankscore=0.927;fathmm-MKL_coding_score=0.900;fathmm-MKL_coding_rankscore=0.506;fathmm-MKL_coding_pred=D;Eigen_coding_or_noncoding=c;Eigen-raw=0.461;Eigen-PC-raw=0.469;GenoCanyon_score=1.000;GenoCanyon_score_rankscore=0.747;integrated_fitCons_score=0.672;integrated_fitCons_score_rankscore=0.522;integrated_confidence_value=0;GERP++_RS=4.22;GERP++_RS_rankscore=0.490;phyloP100way_vertebrate=4.989;phyloP100way_vertebrate_rankscore=0.634;phyloP20way_mammalian=1.047;phyloP20way_mammalian_rankscore=0.674;phastCons100way_vertebrate=1.000;phastCons100way_vertebrate_rankscore=0.715;phastCons20way_mammalian=0.999;phastCons20way_mammalian_rankscore=0.750;SiPhy_29way_logOdds=17.151;SiPhy_29way_logOdds_rankscore=0.866;Interpro_domain=.;GTEx_V6_gene=ENSG00000162415.6;GTEx_V6_tissue=Nerve_Tibial;esp6500siv2_all=0.0560;esp6500siv2_aa=0.0160;esp6500siv2_ea=0.0761;ExAC_ALL=0.0553;ExAC_AFR=0.0140;ExAC_AMR=0.0386;ExAC_EAS=0.0005;ExAC_FIN=0.0798;ExAC_NFE=0.0788;ExAC_OTH=0.0669;ExAC_SAS=0.0145;ExAC_nontcga_ALL=0.0541;ExAC_nontcga_AFR=0.0129;ExAC_nontcga_AMR=0.0379;ExAC_nontcga_EAS=0.0004;ExAC_nontcga_FIN=0.0798;ExAC_nontcga_NFE=0.0802;ExAC_nontcga_OTH=0.0716;ExAC_nontcga_SAS=0.0144;ExAC_nonpsych_ALL=0.0496;ExAC_nonpsych_AFR=0.0140;ExAC_nonpsych_AMR=0.0386;ExAC_nonpsych_EAS=0.0005;ExAC_nonpsych_FIN=0.0763;ExAC_nonpsych_NFE=0.0785;ExAC_nonpsych_OTH=0.0638;ExAC_nonpsych_SAS=0.0145;1000g2015aug_all=0.024361;1000g2015aug_afr=0.0038;1000g2015aug_amr=0.0461;1000g2015aug_eur=0.0795;1000g2015aug_sas=0.0041;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=rs33931638;avsnp150=rs33931638;CADD13_RawScore=4.716301;CADD13_PHRED=24.6;Eigen=0.4614;REVEL=0.098;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0507;gnomAD_genome_AFR=0.0114;gnomAD_genome_AMR=0.0430;gnomAD_genome_ASJ=0.1159;gnomAD_genome_EAS=0;gnomAD_genome_FIN=0.0802;gnomAD_genome_NFE=0.0702;gnomAD_genome_OTH=0.0695;gerp++gt2=4.22;cosmic70=.;InterVar_automated=Benign;PVS1=0;PS1=0;PS2=0;PS3=0;PS4=0;PM1=0;PM2=0;PM3=0;PM4=0;PM5=0;PM6=0;PP1=0;PP2=0;PP3=0;PP4=0;PP5=0;BA1=1;BS1=1;BS2=0;BS3=0;BS4=0;BP1=0;BP2=0;BP3=0;BP4=0;BP5=0;BP6=0;BP7=0;Kaviar_AF=0.0552127;Kaviar_AC=8536;Kaviar_AN=154602;ALLELE_END"

expected_solution = {
    'thousandg2015aug_all': '0.024361', 
    'ExAC_ALL': '0.0553', 
    'SIFT_pred': 'D', 
    'Polyphen2_HDIV_pred': 'P', 
    'Polyphen2_HVAR_pred': 'P', 
    'LRT_pred': 'N',
    'MutationTaster_pred': 'D', 
    'MutationAssessor_pred': 'L', 
    'FATHMM_pred': 'T', 
    'PROVEAN_pred': 'N', 
    'MetaSVM_pred': 'T', 
    'MetaLR_pred': 'T', 
    'fathmm_MKL_coding_pred': 'D'
}


solution  = pull_info_values(sample_info_input)
assert solution == expected_solution


# Part 7 (10 points)

Remember that to insert a record in SQLite, you have to use `cur.execute(sql, values)`, where `sql` is the insert statement and `values` is a list/tuple of values that will be substituted into the `sql` string wherever there is a question mark. 

Write a function that takes in as input: CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values and returns a list with the values in the following order:
```
CHROM 
POS
ID
REF
ALT
QUAL
FILTER
thousandg2015aug_all # 1000 has been replaced by the text thousand
ExAC_ALL
FATHMM_pred
LRT_pred
MetaLR_pred
MetaSVM_pred
MutationAssessor_pred
MutationTaster_pred
PROVEAN_pred
Polyphen2_HDIV_pred
Polyphen2_HVAR_pred
SIFT_pred
fathmm_MKL_coding_pred # note that the dash has been replaced by underscore
```

The info_values dictionary contains the predictor values. Use `None` for any empty/missing value for info fields, thousandg2015aug_all, and ExAC_ALL. 



In [21]:
def build_values_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values):
    """
    See part 7 description 
    
    """
    # YOUR CODE HERE
    build_values = [CHROM, POS, ID, REF, ALT, QUAL, FILTER]
    
    temp = 'thousandg2015aug_all'
    if temp in info_values.keys():
        if info_values[temp] == '' or info_values[temp] == '.':
                info_values[temp] = None 
        build_values.append(info_values[temp])
    else : build_values.append(None)
    temp = 'ExAC_ALL'
    if temp in info_values.keys():
        if info_values[temp] == '' or info_values[temp] == '.':
                info_values[temp] = None 
        build_values.append(info_values[temp])
    else : build_values.append(None) 
        
    key_list = [i for i in predictor_fk_map.keys()]
    key_list.sort()
    for key in key_list:
        if key in info_values:

            if '-' in key :
                key = key.replace('-','_')
            if info_values[key] in predictor_fk_map[key].keys() :
                build_values.append(predictor_fk_map[key][info_values[key]])
            else : build_values.append(None)
        else : build_values.append(None)
            
    return build_values
        
        
        
    
#     raise NotImplementedError()



In [22]:
CHROM, POS, ID, REF, ALT, QUAL, FILTER = (7, 87837848, '.', 'C', 'A', 418.25, 'PASS') 
info_values = {'SIFT_pred': 'D', 'Polyphen2_HDIV_pred': 'D', 'Polyphen2_HVAR_pred': 'D', 'LRT_pred': 'D', 'MutationTaster_pred': 'D', 'MutationAssessor_pred': 'H', 'FATHMM_pred': 'T', 'PROVEAN_pred': 'D', 'MetaSVM_pred': 'D', 'MetaLR_pred': 'D', 'fathmm_MKL_coding_pred': 'D'}

results = build_values_list(CHROM, POS, ID, REF, ALT, QUAL, FILTER, info_values)
expected_results = [7, 87837848, '.', 'C', 'A', 418.25, 'PASS', None, None, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1]
assert results == expected_results


# Part 8 (No Points)

Create a function that takes the database `conn` and `values` from the `build_values_list` function to insert a variant record. 

IMPORTANT: Function should return the id of the row inserted, which will be VariantId


In [23]:
def insert_variant(conn, values):
    """
    See description Part 8
    """
    # YOUR CODE HERE
    with conn :
        cur = conn.cursor()
        cur.execute('''SELECT VariantID FROM Variants;''')
        ivalues = [len(cur.fetchall()) + 1]
        ivalues.extend(values)
        insertsql = 'INSERT INTO Variants VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
        cur = conn.cursor()
        cur.execute(insertsql,ivalues)
        c = cur.lastrowid
    return c
#     raise NotImplementedError()

In [24]:
# db_file = 'lab2.db'
# conn = create_connection(db_file)
# insert_variant(conn,results)
# with conn :
#     a = '''SELECT * FROM Variants;'''
#     cur = conn.cursor()
#     cur.execute(a)
#     rows = cur.fetchall()
#     for row in rows:
#         print(row)  

# Part 9 (No Points)

Create a function that takes the database `conn` and `values` which is the tuple `(VariantId, PredictorName, PredictorValue)` Where `VariantId` will be the return value from the `insert_variant` function,  `PredictorName` is the name of the predictor, and `PredictorValue` is the mapping of the text value to a numeric value. 


In [25]:
def insert_predictionstat(conn, values):
    """
    See description in part 9
    """
    # YOUR CODE HERE
    with conn :
        cur = conn.cursor()
        cur.execute('''SELECT VariantID FROM PredictionStats;''')
        ivalues = [len(cur.fetchall()) + 1]
        ivalues.extend(values)
        insertsql = 'INSERT INTO PredictionStats VALUES (?, ?, ?, ?)'
        cur = conn.cursor()
        cur.execute(insertsql,ivalues)
    
#     raise NotImplementedError()


# Part 10 (No Points)

Write a function to insert records into both the variants and predictor_stats table. 
Hint:
1) Open connection to database  
2) Read file one line at a time using gzip read
3) Extract CHROM, POS, ID, REF, ALT, QUAL, FILTER, INFO
4) Use the pull_info_values function
5) Use build_values_list 
6) Use insert_variant -- save variant_id
7) Use insert_predictionstat -- insert each predictor at a time and remember to use `prediction_mapping` mapping. 




```

prediction_mapping = {
    'FATHMM_pred': {'T': 0, 'D': 1},
    'MetaLR_pred': {'T': 0, 'D': 1},
    'MetaSVM_pred': {'T': 0, 'D': 1},
    'SIFT_pred': {'T': 0, 'D': 1},
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 0},
    'LRT_pred': {'U': 0, 'N': 0, 'D': 1},
    'MutationAssessor_pred': {'H': 1, 'N': 0, 'L': 0.25, 'M': 0.5},  
    'MutationTaster_pred': {'D': 1, 'P': 0, 'N': 0, 'A': 1},
    'PROVEAN_pred': {'D': 1, 'N': 0},
    'Polyphen2_HDIV_pred': {'D': 1, 'B': 0, 'P': 0.5},
    'Polyphen2_HVAR_pred': {'D': 1, 'B': 0, 'P': 0.5},
}


```


In [26]:
prediction_mapping = {
    'FATHMM_pred': {'T': 0, 'D': 1},
    'MetaLR_pred': {'T': 0, 'D': 1},
    'MetaSVM_pred': {'T': 0, 'D': 1},
    'SIFT_pred': {'T': 0, 'D': 1},
    'fathmm_MKL_coding_pred': {'D': 1, 'N': 0},
    'LRT_pred': {'U': 0, 'N': 0, 'D': 1},
    'MutationAssessor_pred': {'H': 1, 'N': 0, 'L': 0.25, 'M': 0.5},  
    'MutationTaster_pred': {'D': 1, 'P': 0, 'N': 0, 'A': 1},
    'PROVEAN_pred': {'D': 1, 'N': 0},
    'Polyphen2_HDIV_pred': {'D': 1, 'B': 0, 'P': 0.5},
    'Polyphen2_HVAR_pred': {'D': 1, 'B': 0, 'P': 0.5},
}


def populate_variants_predictorstats_tables(db_file, filename):
    """
    See description in part 10
    """
    # YOUR CODE HERE
    conn = create_connection(db_file)
    with gzip.open(filename,'rt') as fp :
        header=[]
        line=[]
        for l in fp:
            if l.startswith('##'):
                continue # lines starting with double hash are skipped
            else:
                if l.startswith('#'):
                    header.append(l.strip()[1:])
                else:
                    line.append(l.strip())   
        extract_list = ['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO']
        list_variant_dict=[]
        for i in line:
            dic = dict(zip(header[0].split("\t"), i.split("\t")))
            list_variant_dict.append(dic)
        for j in list_variant_dict:
            dic_info_values = pull_info_values(j['INFO'])
            build_values = build_values_list(j['CHROM'],j['POS'],j['ID'],j['REF'],j['ALT'],j['QUAL'],j['FILTER'],dic_info_values)
            variantid = insert_variant(conn,build_values)
            for k,v in dic_info_values.items() :
                if k in prediction_mapping.keys():
                    if v in prediction_mapping[k].keys():
                        values = (variantid,k,prediction_mapping[k][v])
                        insert_predictionstat(conn, values)
            
    
    
#     raise NotImplementedError()
try:
    conn.close()
except:
    pass

db_file = 'lab2.db'
filename = 'test_4families_annovar.vcf.gz'
populate_variants_predictorstats_tables(db_file, filename)

# Part 11 (10 Points)

Write a function that returns the total number of variants



In [61]:
def num_of_total_variants(conn):
    # YOUR CODE HERE
    with conn:
        cur = conn.cursor()
        cur.execute('''SELECT VariantID FROM Variants;''')
        count = len(cur.fetchall())
    return count
#     raise NotImplementedError()


In [62]:
# a = 'select * from Variants limit 3;'
# cur = conn.cursor()
# cur.execute(a)
# d = cur.fetchall()
# for e in d:
#     print(e)

ProgrammingError: Cannot operate on a closed database.

In [63]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert num_of_total_variants(conn) == 50001
conn.close()

# Part 12 (10 Points)
Write a function returns the total number of variant predictions -- the count of the predictiostats table. 

In [64]:
def num_of_total_variant_predictions(conn):
    """
    Part 12
    """
    # YOUR CODE HERE
    with conn:
        cur = conn.cursor()
        cur.execute('''SELECT PredictorStatsID FROM PredictionStats;''')
        count = len(cur.fetchall())
    return count
#     raise NotImplementedError()



In [65]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert num_of_total_variant_predictions(conn) == 1324
conn.close()

In [66]:
# db_file = 'lab2.db'
# conn = create_connection(db_file)
# a = 'select * from PredictionStats limit 3;'
# cur = conn.cursor()
# cur.execute(a)
# d = cur.fetchall()
# for e in d:
#     print(e)

(1, 614, 'SIFT_pred', 1.0)
(2, 614, 'Polyphen2_HDIV_pred', 1.0)
(3, 614, 'Polyphen2_HVAR_pred', 1.0)


# Part 13 (10 Points)
Return the total number of variant predictions that have value greater than zero. Number of values from the predictionstats table that are greater than 0. 

In [2]:
def num_of_total_variant_predictions_with_value_gt_zero(conn):
    """
    See part 13 description
    """
    # YOUR CODE HERE
    with conn:
        cur = conn.cursor()
        cur.execute('''SELECT PredictorStatsID FROM PredictionStats WHERE PredictorValue > 0;''')
        count = len(cur.fetchall())
    return count
    
#     raise NotImplementedError()



In [3]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert num_of_total_variant_predictions_with_value_gt_zero(conn) == 219
conn.close()

NameError: name 'create_connection' is not defined

# Part 14 (10 Points)

Write a function that given `CHROM, POS, ID, REF, ALT` returns a variant's info with the following columns (column order is important) :
```
Variants.CHROM,
Variants.POS,
Variants.ID,
Variants.REF,
Variants.ALT,
Variants.QUAL,
Variants.FILTER,
Variants.thousandg2015aug_all,
Variants.ExAC_ALL,
FATHMM_pred.prediction,
LRT_pred.prediction,
MetaLR_pred.prediction,
MetaSVM_pred.prediction,
MutationAssessor_pred.prediction,
MutationTaster_pred.prediction,
PROVEAN_pred.prediction,
Polyphen2_HDIV_pred.prediction,
Polyphen2_HVAR_pred.prediction,
SIFT_pred.prediction,
fathmm_MKL_coding_pred.prediction,
sum(PredictionStats.PredictorValue)
```

For the predictions, return the actual text value. And the last column is the sum of all the mapped prediction scores for a given variant!



In [27]:
def fetch_variant(conn, CHROM, POS, ID, REF, ALT):
    """
    See Part 14 description
    """
    # YOUR CODE HERE
    with conn:
        cur = conn.cursor()
        cur.execute('select Variants.CHROM,Variants.POS,Variants.ID,Variants.REF,Variants.ALT,Variants.QUAL,Variants.FILTER,Variants.thousandg2015aug_all,Variants.ExAC_ALL,FATHMM_pred.prediction,LRT_pred.prediction,MetaLR_pred.prediction,MetaSVM_pred.prediction,MutationAssessor_pred.prediction,MutationTaster_pred.prediction,PROVEAN_pred.prediction,Polyphen2_HDIV_pred.prediction,Polyphen2_HVAR_pred.prediction,SIFT_pred.prediction,fathmm_MKL_coding_pred.prediction,sum(PredictionStats.PredictorValue) from Variants LEFT JOIN FATHMM_pred USING(FATHMM_predID) LEFT JOIN LRT_pred USING(LRT_predID) LEFT JOIN MetaLR_pred USING(MetaLR_predID) LEFT JOIN MetaSVM_pred USING(MetaSVM_predID) LEFT JOIN MutationAssessor_pred USING(MutationAssessor_predID) LEFT JOIN MutationTaster_pred USING(MutationTaster_predID) LEFT JOIN PROVEAN_pred USING(PROVEAN_predID) LEFT JOIN Polyphen2_HDIV_pred USING(Polyphen2_HDIV_predID) LEFT JOIN Polyphen2_HVAR_pred USING(Polyphen2_HVAR_predID) LEFT JOIN SIFT_pred USING(SIFT_predID) LEFT JOIN fathmm_MKL_coding_pred USING(fathmm_MKL_coding_predID) LEFT JOIN PredictionStats USING(VariantID) where Variants.CHROM = ?  and Variants.POS = ? and Variants.ID = ? and Variants.REF = ? and Variants.ALT = ? GROUP BY PredictionStats.VariantID ;' , (CHROM, POS,ID, REF, ALT,))
        row = cur.fetchall()
        return row[0]
    
    
    
    
    
#     raise NotImplementedError()
    

In [28]:
# db_file = 'lab2.db'
# conn = create_connection(db_file)
# print(fetch_variant(conn, '5', 155935708, 'rs45559835', 'G', 'A'))

# print('5', 155935708, 'rs45559835', 'G', 'A', 1577.12, 'PASS', 0.0189696, 0.0451, 'D', 'D', 'T', 'T', 'L', 'D', 'D', 'P', 'B', 'T', 'D', 5.75)

('5', 155935708, 'rs45559835', 'G', 'A', 1577.12, 'PASS', 0.0189696, 0.0451, 'D', 'D', 'T', 'T', 'L', 'D', 'D', 'P', 'B', 'T', 'D', 5.75)
5 155935708 rs45559835 G A 1577.12 PASS 0.0189696 0.0451 D D T T L D D P B T D 5.75


In [29]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, '22', 25599849, 'rs17670506', 'G', 'A') == ('22', 25599849, 'rs17670506', 'G', 'A', 3124.91, 'PASS', 0.0251597, 0.0425, 'D', 'D', 'T', 'T', 'M', 'D', 'D', 'D', 'D', 'D', 'D', 8.5)
conn.close()

In [30]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, 'X', 2836184, 'rs73632976', 'C', 'T') == ('X', 2836184, 'rs73632976', 'C', 'T', 1892.12, 'PASS', None, 0.0427, 'D', 'U', 'D', 'T', 'M', 'P', 'D', 'P', 'P', 'D', 'D', 6.5)
conn.close()

In [31]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, '5', 155935708, 'rs45559835', 'G', 'A') == ('5', 155935708, 'rs45559835', 'G', 'A', 1577.12, 'PASS', 0.0189696, 0.0451, 'D', 'D', 'T', 'T', 'L', 'D', 'D', 'P', 'B', 'T', 'D', 5.75)
conn.close()

In [32]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert fetch_variant(conn, '4', 123416186, '.', 'A', 'G') == ('4', 123416186, '.', 'A', 'G', 23.25, 'PASS', None, None, None, None, None, None, None, None, None, None, None, None, None, None)
conn.close()

In [33]:
# db_file = 'lab2.db'
# conn = create_connection(db_file)
# cur = conn.cursor()
# a = 'Variants'
# cur.execute('PRAGMA TABLE_INFO({})'.format(a))
# table_schema = cur.fetchall()
# column_names = [tup[1] for tup in table_schema]
# column_type = [tup[2] for tup in table_schema]
# dic= dict(zip(column_names,column_type))
# print(dic)
# conn = create_connection(db_file)
# cur = conn.cursor()
# b = 'rs17670506'
# o = 'G'
# cur.execute("SELECT * FROM Variants where ID = ? and REF = ? limit 2;",(b,o,))
# oi = cur.fetchall()
# for i in oi :
#     print(i)
# conn.close()

In [34]:
# db_file = 'lab2.db'
# conn = create_connection(db_file)
# print(fetch_variant(conn, '22', 25599849, 'rs17670506', 'G', 'A'))

In [35]:
# db_file = 'lab2.db'
# conn = create_connection(db_file)
# CHROM, POS,ID, REF, ALT = (22, 25599849,'rs17670506', 'G', 'A')
# G = 'G'
# A = 'A'
# # a = 'select * from Variants  WHERE Variants.CHROM = {c} and Variants.POS = {p} and Variants.ID = {i} and \
# #         Variants.REF = {r} and Variants.ALT = {a} GROUP BY PredictionStats.VariantID".format(c=CHROM, p=POS,i=ID,r=REF ,a=ALT) ;'
# cur = conn.cursor()
# # cur.execute('select * from Variants  WHERE Variants.CHROM = {c} and Variants.POS = {p} and Variants.ID = {i} and \
# #         Variants.REF = {r} and Variants.ALT = {a} GROUP BY PredictionStats.VariantID".format(c=CHROM, p=POS,i=ID,r=REF ,a=ALT) ;')
# cur.execute('select Variants.CHROM,Variants.POS,Variants.ID,Variants.REF,Variants.ALT,Variants.QUAL,Variants.FILTER,Variants.thousandg2015aug_all,Variants.ExAC_ALL,FATHMM_pred.prediction,LRT_pred.prediction,MetaLR_pred.prediction,MetaSVM_pred.prediction,MutationAssessor_pred.prediction,MutationTaster_pred.prediction,PROVEAN_pred.prediction,Polyphen2_HDIV_pred.prediction,Polyphen2_HVAR_pred.prediction,SIFT_pred.prediction,fathmm_MKL_coding_pred.prediction,sum(PredictionStats.PredictorValue) from Variants LEFT JOIN FATHMM_pred USING(FATHMM_predID) LEFT JOIN LRT_pred USING(LRT_predID) LEFT JOIN MetaLR_pred USING(MetaLR_predID) LEFT JOIN MetaSVM_pred USING(MetaSVM_predID) LEFT JOIN MutationAssessor_pred USING(MutationAssessor_predID) LEFT JOIN MutationTaster_pred USING(MutationTaster_predID) LEFT JOIN PROVEAN_pred USING(PROVEAN_predID) LEFT JOIN Polyphen2_HDIV_pred USING(Polyphen2_HDIV_predID) LEFT JOIN Polyphen2_HVAR_pred USING(Polyphen2_HVAR_predID) LEFT JOIN SIFT_pred USING(SIFT_predID) LEFT JOIN fathmm_MKL_coding_pred USING(fathmm_MKL_coding_predID) LEFT JOIN PredictionStats USING(VariantID) where Variants.CHROM = ?  and Variants.POS = ? and Variants.ID = ? and Variants.REF = ? and Variants.ALT = ? GROUP BY PredictionStats.VariantID ;' , (CHROM, POS,ID, REF, ALT,))# WHERE Variants.CHROM = {CHROM} and Variants.POS ={POS} and Variants.REF ={REF} and Variants.ALT ={ALT} GROUP BY PredictionStats.VariantID;'.format(CHROM = CHROM, POS=POS, REF=REF, ALT=ALT))

# d = cur.fetchall()
# print(d)
# for e in d:
#     print(e)

# Part 15 (10 Points)
Write a function that returns the variant with the highest predictor score sum. 

Return the variant info the following order:
```
Variants.CHROM,
Variants.POS,
Variants.ID,
Variants.REF,
Variants.ALT,
Variants.QUAL,
Variants.FILTER,
Variants.thousandg2015aug_all,
Variants.ExAC_ALL,
FATHMM_pred.prediction,
LRT_pred.prediction,
MetaLR_pred.prediction,
MetaSVM_pred.prediction,
MutationAssessor_pred.prediction,
MutationTaster_pred.prediction,
PROVEAN_pred.prediction,
Polyphen2_HDIV_pred.prediction,
Polyphen2_HVAR_pred.prediction,
SIFT_pred.prediction,
fathmm_MKL_coding_pred.prediction,
sum(PredictionStats.PredictorValue)
        
```

Again, return the predictor text values and the last column is the sum of the prediction values. 

In [36]:
def variant_with_highest_sum_of_predictor_value(conn):
    """
    See part 15 description 
    """
    # YOUR CODE HERE
    with conn:
        cur = conn.cursor()
        cur.execute('select Variants.CHROM,Variants.POS,Variants.ID,Variants.REF,Variants.ALT,Variants.QUAL,Variants.FILTER,Variants.thousandg2015aug_all,Variants.ExAC_ALL,FATHMM_pred.prediction,LRT_pred.prediction,MetaLR_pred.prediction,MetaSVM_pred.prediction,MutationAssessor_pred.prediction,MutationTaster_pred.prediction,PROVEAN_pred.prediction,Polyphen2_HDIV_pred.prediction,Polyphen2_HVAR_pred.prediction,SIFT_pred.prediction,fathmm_MKL_coding_pred.prediction,sum(PredictionStats.PredictorValue) from Variants LEFT JOIN FATHMM_pred USING(FATHMM_predID) LEFT JOIN LRT_pred USING(LRT_predID) LEFT JOIN MetaLR_pred USING(MetaLR_predID) LEFT JOIN MetaSVM_pred USING(MetaSVM_predID) LEFT JOIN MutationAssessor_pred USING(MutationAssessor_predID) LEFT JOIN MutationTaster_pred USING(MutationTaster_predID) LEFT JOIN PROVEAN_pred USING(PROVEAN_predID) LEFT JOIN Polyphen2_HDIV_pred USING(Polyphen2_HDIV_predID) LEFT JOIN Polyphen2_HVAR_pred USING(Polyphen2_HVAR_predID) LEFT JOIN SIFT_pred USING(SIFT_predID) LEFT JOIN fathmm_MKL_coding_pred USING(fathmm_MKL_coding_predID) LEFT JOIN PredictionStats USING(VariantID) GROUP BY PredictionStats.VariantID HAVING SUM(PredictionStats.PredictorValue) = (SELECT MAX(sum) FROM (SELECT SUM(PredictionStats.PredictorValue) as sum FROM Variants LEFT JOIN PredictionStats USING(VariantID) GROUP BY PredictionStats.VariantID));')
        row = cur.fetchall()
        return row[0]
 
        
#     raise NotImplementedError()



In [37]:
db_file = 'lab2.db'
conn = create_connection(db_file)
assert variant_with_highest_sum_of_predictor_value(conn) == ('7', 87837848, '.', 'C', 'A', 418.25, 'PASS', None, None, 'T', 'D', 'D', 'D', 'H', 'D', 'D', 'D', 'D', 'D', 'D', 10.0)
conn.close()